In [ ]:
## Recursive problems around trees
Given a binary tree, find its height

In [25]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

def height(node):
    if node is None:
        return 0
    
    left_height = height(node.left)
    right_height = height(node.right)
    
    return max(left_height, right_height) + 1

t = Node(
    value=2, 
    left=Node(3, Node(4)), 
    right=Node(5, Node(6), Node(7))
)
height(t)

3

In [ ]:
Given a binary tree, return the value of the nodes at a given level. 

For example given the tree above,

```

In [24]:
def bfs_at_level(tree, visited=[], level=1):
    if tree is None:
        return
    
    if level == 1:
        visited.append(tree.value)
    elif level > 1:
        bfs_at_level(tree.left, visited, level - 1)
        bfs_at_level(tree.right, visited, level - 1)
    return visited

3

In [20]:
# Depth-first traversals 

def inOrderTraversal(node, results=[]):
    """Visit left-most child, then root, then right"""
    if node is None:
        return
    inOrderTraversal(node.left)
    results.append(node.value)
    inOrderTraversal(node.right)
    return results

def preOrderTraversal(node, results=[]):
    """Visit root first, then left, right"""
    if node is None:
        return
    results.append(node.value)
    preOrderTraversal(node.left)
    preOrderTraversal(node.right)
    return results

def serialize(root):
    if root is None:
        return "#"
    return f"{root.value} {serialize(root.left)} {serialize(root.right)}"

def xx(node):
    if node is None:
        return ""
    return f"{xx(node.left)}{node.value}-{xx(node.right)}"

In [21]:
tree = Node(
    value=2, 
    left=Node(3, Node(4)), 
    right=Node(5, Node(7), Node(6)))
inOrderTraversal(tree)
serialize(tree)

'2 3 4 # # # 5 7 # # 6 # #'

In [4]:

def dfs_helper(node, depth, result=0):
    result += depth
    if node.left is not None:
        result += dfs_helper(node.left, depth + 1)
    if node.right is not None:
        result += dfs_helper(node.right, depth + 1)
    return result
        
def sum_depths(root):
    return dfs_helper(root, depth=0)
    

In [5]:
tree = Node(
    1, 
    Node(2, Node(4, Node(8), Node(9)), Node(5)), 
    Node(3, Node(6), Node(7))
)
sum_depths(tree)

16

In [279]:
def dfs_rooted(node):
    stack = [node]
    total = 0
    while stack:
        current_root = stack.pop(0)
        total += dfs_helper(current_root, depth=0)
        if current_root.left is not None:
            stack.append(current_root.left)
        if current_root.right is not None:
            stack.append(current_root.right)
    return total

In [280]:
dfs_rooted(tree)

26

In [328]:
def bottom_up_dfs(node, result):
    # pair.first = the number of nodes in a subtree
    # pair.second = the sum of depths of that subtree
    pair = [1, 0]
    
    if node.left is not None:
        child_pair = bottom_up_dfs(node.left, result)
        pair[1] += child_pair[1] + child_pair[0]
        pair[0] += child_pair[0]
        result += pair[1]
        
    if node.right is not None:
        child_pair = bottom_up_dfs(node.right, result)
        pair[1] += child_pair[1] + child_pair[0]
        pair[0] += child_pair[0]
        result += pair[1]
    
    print(pair[0], pair[1], result)
    return pair[0], pair[1], result
    
def rooted_depths(node):
    _,_,result = bottom_up_dfs(node, result=0)
    return result
    

In [329]:
rooted_depths(tree)

1 0 0
1 0 1
3 2 3
1 0 5
5 6 11
1 0 11
1 0 12
3 2 14
9 16 27


27